In [61]:
import os
import pandas as pd
from modules.util import numerical_to_float


df1 = pd.read_csv('data/data_files/BG_meteo_30min_20230101_20230801.csv', sep=',', na_values=['NaN']).drop(0)
df2 = pd.read_csv('data/data_files/GW_meteo_30min_20230101_20230801.csv', sep=',', na_values=['NaN']).drop(0)
df3 = pd.read_csv('data/data_files/BG_meteo_30min_20240401_20240608.csv', sep=';', na_values=['NaN']).drop(0)
df4 = pd.read_csv('data/data_files/GW_meteo_30min_20240401_20240608.csv', sep=';', na_values=['NaN']).drop(0)

df1 = df1.drop(["TIMESTAMP_MITTE", "TIMESTAMP_ENDE"], axis=1)
df2 = df2.drop(["TIMESTAMP_MITTE", "TIMESTAMP_ENDE"], axis=1)
df3 = df3.drop(["TIMESTAMP_MITTE", "TIMESTAMP_ENDE"], axis=1)
df4 = df4.drop(["TIMESTAMP_MITTE", "TIMESTAMP_ENDE"], axis=1)

df1_cols = list(df1.columns)
df2_cols = list(df2.columns)
df3_cols = list(df3.columns)
df4_cols = list(df4.columns)

df1_cols.remove("TIMESTAMP_START")
df2_cols.remove("TIMESTAMP_START")
df3_cols.remove("TIMESTAMP_START")
df4_cols.remove("TIMESTAMP_START")



df1 = numerical_to_float(df1, df1_cols)
df2 = numerical_to_float(df2, df2_cols)
df3 = numerical_to_float(df3, df3_cols)
df4 = numerical_to_float(df4, df4_cols)

In [62]:
# merge 2023 and 2024 data, respectively
df_23 = df1.merge(df2, how="left", on="TIMESTAMP_START")
df_24 = df3.merge(df4, how="left", on="TIMESTAMP_START")

# concat both
df = pd.concat([df_23, df_24])

# merge some cols
df['kurzwEinstrahlung'] = df['kurzwEinstrahlung_300cm'].combine_first(df['kurzwEinstrahlung_43m'])
df['Bodentemp_1_30cm_temp'] = df['Bodentemp_1_30cm_x'].combine_first(df['Bodentemp_1_30cm_y'])
df['Bodentemp_1_30cm_temp'] = df['Bodentemp_1_30cm_temp'].combine_first(df['Bodentemp_1_30cm'])
df['Bodentemp_1_30cm'] = df['Bodentemp_1_30cm_temp'].combine_first(df['Bodentemp_30cm_1'])

In [63]:
def nan_percentage(df):
    """
    Returns a DataFrame containing the percentage of NaN values in each column of the input DataFrame.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame.
    
    Returns:
    pd.DataFrame: A DataFrame with columns 'Column' and 'NaN_Percentage' showing the percentage of NaN values for each column.
    """
    # Calculate the percentage of NaN values for each column
    nan_percentage = df.isna().mean() * 100
    
    # Create a new DataFrame with the results
    nan_df = pd.DataFrame({
        'Column': nan_percentage.index,
        'NaN_Percentage': nan_percentage.values
    })
    
    return nan_df

In [64]:
df_nan = nan_percentage(df)
df_nan.to_csv("nans.csv")

In [65]:
df["kurzwEinstrahlung"]

0      -3.695000
1      -4.819000
2      -4.449333
3      -3.713000
4      -4.892000
          ...   
3259   -4.898667
3260   -4.613333
3261   -4.425667
3262   -4.224667
3263   -4.180000
Name: kurzwEinstrahlung, Length: 13440, dtype: float64